# **Introduction**

In the tutorials, we are going to look at an example of image classification using Keras and Tensorflow. The dataset which we are going to work on is CIFAR-10. CIFAR-10 is a large image dataset containing over 60,000 images representing 10 different classes of objects like cats, planes, and cars. 

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils
# Set random seed for purposes of reproducibility
seed = 21

from keras.datasets import cifar10

Loading and preparing data:

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

Now, we want to build the CNN model. Keras has different formats to build models but "Sequential" is the most commonly used. 

The first layer is a a convolutional layer. This layer runs convolutional filter on its input.
We need to specify the number of filters(32) and the size of the filter (3*3). Also, the input shape for creating the first layer and activation and padding are needed. We wont change the size of the images here so the padding is set to "same".

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

To prevet overfitting, a dropout layer is also needed. Dropout randomly eliminates some of the connections between the layers. If set to 0.2, 20% of tje existing connections will be dropped. Batchnormalization can also be used to make sure our network creates the activations with the same distribution that we desire.

In [ ]:
model.add(Dropout(0.2))
model.add(BatchNormalization())

Now, there will be another layer. This time, we use higher neurons to help with learning more complex representations.
Pooling layer can also be added to make the classification more robust so it can learn relevant patterns. We again add the dropout and batchnormalization.

The number of convolutional layers can vary based on your preference. Although, it will have computational cost. Please note that increment of neurons in convolutional layers help model learn more complex representations (It is advised to make them powers of 2 to have a slight benefit when training a GPU).

We also dont want pooling layers. They discard some data and this is possible that not enough data is left for the densely connected layers.  

Now, we can add more of these layers to give the network more representations:

In [ ]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

Data need to be flattened after the convolutional layers and then a dropout to prevet overfitting.

In [ ]:
model.add(Flatten())
model.add(Dropout(0.2))

Now, the densely connected layers will be added.
The kernel constraint (here: maxnorm) can regularize the data as it learns. This helps to prevent overfitting. 

In [ ]:
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

Finally, the number of classes will selected to be the number of neurons in the final layer.  Then, the softmax activation function selects the neuron with the highest probability as its output.

After the model is designed, there is a need for the optimizer to tune the weights in netword to approach the lowest loss. The Adam algorithm is chosen that is one of the most commonly used optimizers with high performances.


In [ ]:
model.add(Dense(class_num))
model.add(Activation('softmax'))
epochs = 25
optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_37 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_33 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_38 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)       

At the next step, we train the model using 50000 samples and validatinf on 10000 samples.

In [ ]:
numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

Epoch 1/25
782/782 [==============================] - 401s 510ms/step - loss: 1.7890 - accuracy: 0.3817 - val_loss: 1.1074 - val_accuracy: 0.6030
Epoch 2/25
782/782 [==============================] - 391s 500ms/step - loss: 1.0851 - accuracy: 0.6165 - val_loss: 0.8567 - val_accuracy: 0.6939
Epoch 3/25
782/782 [==============================] - 400s 512ms/step - loss: 0.8674 - accuracy: 0.6951 - val_loss: 0.8006 - val_accuracy: 0.7178
Epoch 4/25
782/782 [==============================] - 402s 514ms/step - loss: 0.7653 - accuracy: 0.7304 - val_loss: 0.7826 - val_accuracy: 0.7218
Epoch 5/25
782/782 [==============================] - 399s 511ms/step - loss: 0.6992 - accuracy: 0.7542 - val_loss: 0.6780 - val_accuracy: 0.7646
Epoch 6/25
782/782 [==============================] - 402s 515ms/step - loss: 0.6518 - accuracy: 0.7710 - val_loss: 0.6077 - val_accuracy: 0.7831
Epoch 7/25
782/782 [==============================] - 403s 515ms/step - loss: 0.6096 - accuracy: 0.7843 - val_loss: 0.6275 -

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

**Reference:**
[Image Recognition in Python with TensorFlow and Keras](https://stackabuse.com/image-recognition-in-python-with-tensorflow-and-keras/)